In [ ]:
from data_classes.IMDBLightningDataModule import IMDBLightningDataModule
from models.ClassifierSystem import LightningClassifier
from data_classes.pretrained_embeddings import get_pretrained_embeddings
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [ ]:
vocab, vectors = get_pretrained_embeddings()

In [ ]:
imdb_data = IMDBLightningDataModule(vocab)

In [ ]:
classifier = LightningClassifier(vocab=vocab, vectors=vectors, embedding_size=300, learning_rate=1e-3, num_layers=1, hidden_size=100)

In [ ]:
# logger = TensorBoardLogger('exp_logs', name='lstm')

In [ ]:
trainer = Trainer(max_epochs=100, gpus=1, auto_select_gpus=True, auto_scale_batch_size=False, auto_lr_find=False, callbacks=[EarlyStopping(monitor="Val Loss", patience=3)])
# , track_grad_norm=2, accumulate_grad_batches=8, logger=[logger]

In [ ]:
trainer.test(classifier, imdb_data)

In [ ]:
# trainer.tune(classifier, imdb_data)

In [ ]:
# classifier.batch_size

In [ ]:
classifier.learning_rate

In [ ]:
trainer.fit(classifier, imdb_data)

In [ ]:
trainer.test(classifier, imdb_data)

In [ ]:
# Run learning rate finder
# lr_finder = trainer.tuner.lr_find(classifier, imdb_data)

In [ ]:
# Plot with
# fig = lr_finder.plot(suggest=True)
# fig.show()

In [ ]:
# Pick point based on plot, or get suggestion
# new_lr = lr_finder.suggestion()
# new_lr